In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import norm

In [ ]:
def show_vol_smile():
  F=0.03
  strikes = np.linspace(0.02, 0.04, 15)
  market_vols=[]
  base_vol = 0.25

  for K in strikes:
    moneyness=np.log(K/F)

    smile_curvature=0.4*moneyness**2
    skew_asymmetry=-0.15*moneyness
    vol=base_vol+smile_curvature+skew_asymmetry
    market_vols.append(vol)

  plt.plot([k*100 for k in strikes], [v*100 for v in market_vols],
           "bo-", linewidth=3)
  plt.axvline(F*100, color="green", linestyle=":", label="B-S Prediction")
  plt.axhline(y=25, color="red", linestyle="--", label="ATM")
  plt.xlabel("Strike")
  plt.ylabel("Volatility")
  plt.grid(True)
  plt.legend()
  plt.show()
  vol_range = max(market_vols) - min(market_vols)
  print(vol_range)

  itm_vol=market_vols[2]
  otm_vol=market_vols[-3]
  skew_magnitude=abs(itm_vol-otm_vol)
  print(skew_magnitude)

In [ ]:
show_vol_smile()


In [ ]:
class SABRParameters:
  def __init__(self, alpha:float=0.2, beta:float=0.5, 
               rho:float=-0.3, sigma:float=0.25):
    assert alpha > 0, "alpha must be greater than 0"
    assert beta > 0, "beta must be greater than 0"
    assert rho >= -1 and rho <= 1, "rho must be between -1 and 1"
    assert sigma > 0, "sigma must be greater than 0"
    self.alpha = alpha
    self.beta = beta
    self.rho = rho
    self.sigma = sigma

    



In [ ]:
params = SABRParameters(alpha=0.3, beta=0.5, rho=-0.4, sigma=0.25)

In [ ]:
def simulate_sabr_dynamics(F0:float, T:int, params:SABRParameters, 
                           n_path:int=2000, n_steps:int=100, 
                           F_clip:float=0.001, sigma_clip:int=0.01):
  dt = T/n_steps
  F_paths = np.zeros((n_path, n_steps+1))
  sigma_paths = np.zeros((n_path, n_steps+1))
  F_paths[:, 0] = F0
  sigma_paths[:,0] = params.sigma

  for step in range(n_steps):
    W = np.random.normal(size=n_path)
    Z = params.rho*W+np.sqrt(1-params.rho**2)*np.random.normal(size=n_path)

    F_curr = F_paths[:, step]
    sigma_curr = sigma_paths[:, step]

    dF = sigma_curr+(F_curr**params.beta)*np.sqrt(dt)*W

    dsigma = params.alpha*sigma_curr*np.sqrt(dt)*Z

    F_paths[:, step+1] = np.maximum(F_curr+dF, F_clip)
    sigma_paths[:, step+1] = np.maximum(sigma_curr+dsigma, sigma_clip)

    return F_paths, sigma_paths



In [ ]:
F0 = 0.03
T = 1.0
F_paths, sigma_paths = simulate_sabr_dynamics(F0, T, params)
n = np.linspace(0, T, F_paths.shape[1])

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes[0, 0].plot(n, F_paths[:50].T * 100, alpha=0.1, color="blue")
axes[0, 0].plot(n, np.mean(F_paths, axis=0)*100, "r-", linewidth=3, label="F Mean")
axes[0, 0].legend()
axes[0, 0].grid(True)

axes[0, 1].plot(n, sigma_paths[:50].T * 100, alpha=0.1, color="blue")
axes[0, 1].plot(n, np.mean(sigma_paths, axis=0)*100, "r-", linewidth=3, label="sigma Mean")
axes[0, 1].legend()
axes[0, 1].grid(True)

dF = np.diff(F_paths, axis=1)
dsigma = np.diff(sigma_paths, axis=1)
sample = np.random.choice(len(dF), 2000)
axes[1, 0].scatter(dsigma[sample], dF[sample] * 100, alpha=0.1, s=1)
axes[1, 0].grid(True)
axes[1, 0].set_xlabel("dsigma")
axes[1, 0].set_ylabel("dF")
emp_corr = np.corrcoef(dsigma[sample], dF[sample])[0, 1]
print(f"Theory: {params.rho}")
print(f"Emiprical: {emp_corr}")

axes[1, 1].hist(F_paths[:, -1], bins=30, alpha=0.7, density=True)
axes[1, 1].hist(sigma_paths[:, -1], bins=30, alpha=0.7, density=True)
axes[1, 1].grid(True)
axes[1, 1].set_xlabel("Value")
axes[1, 1].set_ylabel("Final Dist")